In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics

In [2]:
from PIL import Image
import cv2 
import os 

In [3]:
import random
import time

In [4]:
wed_data = pd.read_pickle('Wednesday_Packet_Level_Data.pkl')

In [5]:
df = wed_data.copy()

In [6]:
df = df[df['Class Label'] == 0.0]

In [7]:
df = df.reset_index(drop=True)

In [8]:
df.head(5)

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499254962.448456000,192.168.10.14,40.83.143.209,49460,443,1,1,00c1b114eb31b8ac6f3607ee0800,4500013d009f400080067641c0a80a0e28538fd1,c13401bb3be0755575a579ec5018010201c10000,1703030110f19e9b23db6b8b319b6bbf387a8beadf208f...,fwd,0.0,0.0
1,1499254962.448560000,192.168.10.14,40.83.143.209,49460,443,1,278,00c1b114eb31b8ac6f3607ee0800,450005c800a04000800671b5c0a80a0e28538fd1,c13401bb3be0766a75a579ec50100102cadb0000,1703030e308a23eb8dc1f99b3718cf452e9db49a86e8d8...,fwd,0.0,0.0
2,1499254962.448610000,192.168.10.14,40.83.143.209,49460,443,1,1718,00c1b114eb31b8ac6f3607ee0800,450008bd00a1400080066ebfc0a80a0e28538fd1,c13401bb3be07c0a75a579ec501801028b8a0000,5c0cf0c251f7c62c05ca94e31611146e573a54c6f33077...,fwd,0.0,0.0
3,1499254962.992908000,192.168.10.14,40.83.143.209,49460,443,1,3915,00c1b114eb31b8ac6f3607ee0800,4500002800a3400080067752c0a80a0e28538fd1,c13401bb3be0849f75a57b2150100101b7c20000,000000000000,fwd,0.0,0.0
4,1499254963.604097000,192.168.10.12,192.168.10.3,59656,3268,2,1,1866da9be37db8ac6f3604e30800,450001c40e274000400695adc0a80a0cc0a80a03,e9080cc4e2fefcdb3fdca8bb8018012b9e780000010108...,0000018c6082018806092a864886f71201020202011100...,fwd,0.0,0.0


In [9]:
len(df)

8845368

In [10]:
df.shape

(8845368, 14)

In [11]:
df[['Payload']] = df[['Payload']].replace('', '00')

In [12]:
df['Class Label'].value_counts()

0.0    8845368
Name: Class Label, dtype: int64

In [13]:
df.groupby(['Direction', 'Class Label']).size().unstack(fill_value=0)

Class Label,0.0
Direction,
bwd,4829522
fwd,4015846


In [14]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [15]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [16]:
tcp_segment_data_lst = []

for i in range(len(df)):
    x = df['Payload'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 1460):
            output = np.pad(byte_lst, (0, 1460-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:1460].copy())
            

    tcp_segment_data_lst.append(output)
    
df['Payload Array'] = tcp_segment_data_lst

In [17]:
df['pkt_img_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], np.append(x['TCP Header Array'], x['Payload Array']))), axis=1)

In [18]:
df.drop(columns=[ 'Source IP', 'Dst IP', 'Source Port', 'Dst Port', 'Eth Header', 'IP Header',
       'TCP Header', 'Payload', 'IP Header Array', 'TCP Header Array', 'Payload Array'],inplace=True)


In [19]:
new_cols = ['Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_img_values','Binary Label', 'Class Label']
df = df[new_cols]

In [20]:
df['Class Label'].value_counts()

0.0    8845368
Name: Class Label, dtype: int64

In [21]:
df['Stream Index'] = df['Stream Index'].astype(int)

In [22]:
df['Epoch Time'] = df['Epoch Time'].astype(float)

In [23]:
df['Sequence Number'] = df['Sequence Number'].astype('int64')

In [24]:
#df.to_pickle('pkt_image_data_wednesday.pkl')

In [ ]:
for m in range(len(df)):
    df['pkt_img_values'][m] = np.insert(df['pkt_img_values'][m], 0, 0)  

C:\Users\Jalal\AppData\Local\Temp\ipykernel_16976\499766766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pkt_img_values'][m] = np.insert(df['pkt_img_values'][m], 0, 0)


In [ ]:
class_lst = list(df['Class Label'].unique())

In [ ]:
class_lst

In [ ]:
##Slowloris = 1
##slowhttptest = 2
##HULK = 3
#GOLDENEYE = 4
#Heartbleed = 5

In [ ]:
pd.options.mode.chained_assignment = None 

In [ ]:
for i in range(len(class_lst)):
   
    df_class = df[df['Class Label']==class_lst[i]]
    #df_class = df_class.sort_values(by = 'Stream Index')
    print('class category is:', class_lst[i])
    streams_lst = list(df_class['Stream Index'].unique())
    print('Total number of flows in class:', len(streams_lst))
    
    random.shuffle(streams_lst)
    
    #if class_lst[i]==0.0 or class_lst[i]==3.0:
        
       # streams_lst = random.sample(streams_lst, 6700)
       # random.shuffle(streams_lst)
    
    
    
    pkts_per_flow = []
    time_spent_per_class = []

    for j in range(len(streams_lst)):

        df_stream = df_class[df_class['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)

        
        pkts_per_flow.append(len(df_stream))
        
        
        
#         IAT_list = [0]
#         for z in range(len(df_stream)-1):
#             IAT = df_stream['Epoch Time'][z+1] - df_stream['Epoch Time'][z]
    
#             IAT_list.append(IAT)
    
#         minimum = min(IAT_list)
#         maximum = max(IAT_list)
#         IAT_pix_value = [(0 if maximum-minimum == 0 else (round(255*(IAT_list[x] - minimum)/(maximum-minimum)))) for x in range(len(IAT_list))]
        
#         for m in range(len(df_stream)):
#             df_stream['pkt_img_values'][m] = np.insert(df_stream['pkt_img_values'][m], 0, 0)        
        
        

        
        arr = np.zeros((15, 1486, 3), 'uint8')
        red_arr = np.zeros([15, 1486])
        green_arr = np.zeros([15, 1486])
        blue_arr = np.full([15, 1486], 0.0)
        
        for k in range(15):
            os.chdir('C:\\Users\\Jalal\\Desktop\\NIDS Datasets\\CICIDS2017\\Image data 4\\Wednesday_benign')

            
            
            if k >= len(df_stream):
                break
                
            t0 = time.time()
            
            if (df_stream['Direction'][k] =='fwd'): 
                
                 red_arr[k,:] += df_stream['pkt_img_values'][k][:1486]
            
            
            else:
                green_arr[k,:] += df_stream['pkt_img_values'][k][:1486]
            
            
            
            arr[:,:,0] = red_arr
            arr[:,:,1] = green_arr
            arr[:,:,2] = blue_arr

            img = Image.fromarray(arr, 'RGB')

            t1 = time.time()
            total_time_per_image = t1-t0
            
            time_spent_per_class.append(total_time_per_image)
            
            
            image_name = "Strm" + "_"+str(streams_lst[j]) + "_"+"img"+ "_"+str(k+1) +"_"+"pkts.png"
            os.makedirs(os.path.join(str(class_lst[i]), str(k+1)), exist_ok=True)
            directory=os.path.join('C:\\Users\\Jalal\\Desktop\\NIDS Datasets\\CICIDS2017\\Image data 4\\Wednesday_benign',(os.path.join(str(class_lst[i]), str(k+1))))
            file_path = os.path.join(directory, image_name)
            os.chdir(directory)
            img.save(file_path)
            

    
    #print("Packets in all flows list % s" %(pkts_per_flow))   
    print("Average of pkts in flows are % s" % (statistics.mean(pkts_per_flow)))
    print("Median of pkts in flows are % s" % (statistics.median(pkts_per_flow)))
    print("Mode of pkts is % s" % (statistics.mode(pkts_per_flow)))
    print("Average time spend per image for all flows in class is %s seconds" % (statistics.mean(time_spent_per_class)))
    print('==============================================')